In [ ]:
#    “Copyright (C) 2024 Mississippi State University.
 
#    This program is free software: you can redistribute it and/or modify
#    it under the terms of the GNU General Public License as published by
#    the Free Software Foundation, either version 3 of the License, or
#    (at your option) any later version.
 
#    This program is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#    GNU General Public License for more details.
 
#    You should have received a copy of the GNU General Public License
#    along with this program.  If not, see <https://www.gnu.org/licenses/>.
 
# To inquire about relicensing, accessing more training data, collaborating with the author, or any general inquiry about the software, please contact Mississippi State University’s Office of Technology Management at otm@msstate.edu, (662) 325-9263.”


#Linear spectral unmixing technique (test for one location)

import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt

# Specify the path to your cropped image
location1_stacked_path = "C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/clip locations stack/location1_stacked.tif"

# Open the cropped image using rasterio
with rasterio.open(location1_stacked_path) as src:
    cropped_image = src.read()
    crs = src.crs  # Get the CRS information from the metadata

vegetation_endmember = all_average_veg_pixel_values[0][1:] 
soil_endmember =  all_average_soil_pixel_values[0][1:]


# Reshape the cropped image to (pixels, bands)
pixels, bands = cropped_image.shape[1] * cropped_image.shape[2], cropped_image.shape[0]
cropped_data_reshaped = cropped_image.transpose(1, 2, 0).reshape(pixels, bands)

# Perform linear spectral unmixing
n_endmembers = 2  # Number of endmembers (vegetation and soil)
endmembers = np.vstack((vegetation_endmember, soil_endmember))

# Calculate the Moore-Penrose pseudoinverse of endmembers
endmembers_pseudo_inv = np.linalg.pinv(endmembers)

# Estimate fractional abundances
abundances = np.dot(cropped_data_reshaped, endmembers_pseudo_inv)
abundances = np.clip(abundances, 0, 1)  # Clip values to the [0, 1] range

# Reshape abundance maps back to image dimensions
abundance_maps = abundances.reshape(cropped_image.shape[1], cropped_image.shape[2], n_endmembers)

# Output directories for soil and vegetation maps
veg_output_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/vegetation abundance maps/'
soil_output_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/soil abundance maps/'

# Create the output directories if they don't exist
os.makedirs(soil_output_directory, exist_ok=True)
os.makedirs(veg_output_directory, exist_ok=True)
    
location_name = "location1"
    
for i in range(n_endmembers):
    plt.figure(figsize=(6, 6))
    plt.imshow(abundance_maps[:, :, i], cmap='viridis')
    plt.title(f'Fractional Abundance - Endmember {i + 1}')
    plt.colorbar(shrink=0.2)

    # Determine the endmember type
    if i == 0:
        endmember_type = 'vegetation'
    else:
        endmember_type = 'soil'

    # Construct the output filename with location information
    output_filename = f'{location_name}_{endmember_type}_abundance_map.png'

    # Construct the full output path
    if endmember_type == 'vegetation':
        output_directory = veg_output_directory
    else:
        output_directory = soil_output_directory

    output_path = os.path.join(output_directory, output_filename)

    # Save the abundance map as an image
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

print("Abundance maps saved in separate directories.")

In [ ]:
#loop over all the stakced images in the directory to perform LUM  
#Linear spectral unmixing

import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt

# Directory containing TIFF images
image_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/clip locations stack/'

# Output directories for soil and vegetation maps
veg_output_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/vegetation abundance maps/'
soil_output_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/soil abundance maps/'

# Create the output directories if they don't exist
os.makedirs(soil_output_directory, exist_ok=True)
os.makedirs(veg_output_directory, exist_ok=True)


vegetation_abundance_maps = []
soil_abundance_maps = []

# Loop through TIFF files in the directory
for i in range(1, 241):  # Assuming you have images from location1_stacked to location240_stacked
    image_path = os.path.join(image_directory, f'location{i}_stacked.tif')

    # Check if the file exists
    if os.path.exists(image_path):
        # Open the cropped image using rasterio
        with rasterio.open(image_path) as src:
            cropped_image = src.read()
            crs = src.crs  # Get the CRS information from the metadata

        # Determine the location name based on the image filename
        location_name = f'location{i}'

        # Select the appropriate endmembers for the current location
        vegetation_endmember = all_average_veg_pixel_values[i - 1][1:]
        soil_endmember = all_average_soil_pixel_values[i - 1][1:]

        # Reshape the cropped image to (pixels, bands)
        pixels, bands = cropped_image.shape[1] * cropped_image.shape[2], cropped_image.shape[0]
        cropped_data_reshaped = cropped_image.transpose(1, 2, 0).reshape(pixels, bands)

        # Perform linear spectral unmixing
        n_endmembers = 2  # Number of endmembers (vegetation and soil)
        endmembers = np.vstack((vegetation_endmember, soil_endmember))

        # Calculate the Moore-Penrose pseudoinverse of endmembers
        endmembers_pseudo_inv = np.linalg.pinv(endmembers)

        # Estimate fractional abundances
        abundances = np.dot(cropped_data_reshaped, endmembers_pseudo_inv)
        abundances = np.clip(abundances, 0, 1)  # Clip values to the [0, 1] range

        # Reshape abundance maps back to image dimensions
        abundance_maps = abundances.reshape(cropped_image.shape[1], cropped_image.shape[2], n_endmembers)
        
        # Append the vegetation abundance map to the list
        vegetation_abundance_maps.append(abundance_maps[:, :, 0])  # Index 0 for vegetation

        # Append the vegetation abundance map to the list
        soil_abundance_maps.append(abundance_maps[:, :, 1])  # Index 1 for soil


        for j in range(n_endmembers):
            plt.figure(figsize=(6, 6))
            plt.imshow(abundance_maps[:, :, j], cmap='viridis')
            plt.title(f'Fractional Abundance - Endmember {j + 1}')
            plt.colorbar(shrink=0.2)

            # Determine the endmember type
            if j == 0:
                endmember_type = 'vegetation'
            else:
                endmember_type = 'soil'

            # Construct the output filename with location information
            output_filename = f'{location_name}_{endmember_type}_abundance_map.png'

            # Construct the full output path
            if endmember_type == 'vegetation':
                output_directory = veg_output_directory
            else:
                output_directory = soil_output_directory

            output_path = os.path.join(output_directory, output_filename)

            # Save the abundance map as an image
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            plt.close()

print("Abundance maps saved in separate directories.")


In [ ]:
#Create vegetation reflectivity map using the spatial and spectral feature map

import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio

# Directory containing TIFF images for all locations
image_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/clip locations stack/'

# Output directory for vegetation reflectivity maps
output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/vegetation reflectivity maps/'

# Create the output parent directory if it doesn't exist
os.makedirs(output_parent_directory, exist_ok=True)

# Initialize an empty list to store vegetation reflectivity maps for all locations
vegetation_reflectivity_maps = []

# Loop through all locations (assuming you have images from location1_stacked to location240_stacked)
for location_index in range(1, 241):
    location_name = f'location{location_index}'  # Location name based on the index
    
    # Specify the path to your cropped image for the current location
    location_stacked_path = os.path.join(image_directory, f'{location_name}_stacked.tif')

    # Open the cropped image using rasterio
    with rasterio.open(location_stacked_path) as src:
        cropped_image = src.read()
        crs = src.crs  # Get the CRS information from the metadata

    # Load the spectral reflectance signature of pure veg (can replace with your actual veg spectrum)
    vegetation_endmember = all_average_veg_pixel_values[location_index - 1][1:]

    # Access the veg abundance msop
    veg_abundance_map = vegetation_abundance_maps[location_index - 1]  # Veg is the first endmember (index 0)

    # Create a directory for vegetation reflectivity maps
    output_directory = os.path.join(output_parent_directory, location_name)
    os.makedirs(output_directory, exist_ok=True)
    
    # Initialize an empty list to store reflectivity maps for the current location
    location_veg_reflectivity_maps = []

    # Loop through each band in your stacked image
    band_names = ['Blue', 'Green', 'Red', 'Red Edge', 'NIR']  # Replace with actual band names

    for band_index, band_name in enumerate(band_names):
        # Calculate the reflectivity for the current band using abundance and endmember spectrun
        reflectivity_band_veg = vegetation_endmember[band_index] * veg_abundance_map
        
        # Append the reflectivity map to the list
        location_veg_reflectivity_maps.append(reflectivity_band_veg)

        # Plot and save the reflectivity map for the current band
        plt.figure(figsize=(8, 6))
        plt.imshow(reflectivity_band_veg, cmap='viridis')
        plt.colorbar(label='Reflectivity', shrink=0.2)
        plt.title(f'{band_name} Reflectivity Map for {location_name}')
        plt.axis('off')

        # Save the reflectivity map as an image
        output_filename = f'{location_name}_{band_name}_reflectivity_map.png'
        output_path = os.path.join(output_directory, output_filename)
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()
 
    # Append the reflectivity maps for the current location to the list of all maps
    vegetation_reflectivity_maps.append(location_veg_reflectivity_maps)


print("All vegetation reflectivity maps saved.")


In [ ]:
#sanity check for the veg reflectivity map (start with location 0-239)

# Access the reflectivity maps for location 1 from the list of all maps
location1_veg_reflectivity_maps = vegetation_reflectivity_maps[0]

# Check the shape of each reflectivity map for location 1
for band_index, reflectivity_map in enumerate(location1_veg_reflectivity_maps):
    print(f"Shape of Veg Reflectivity Map for Location 1 - Band {band_index + 1}: {reflectivity_map.shape}")

# Plot the reflectivity maps for each band
band_names = ['Blue', 'Green', 'Red', 'Red Edge', 'NIR']  # Replace with actual band names

for band_index, band_name in enumerate(band_names):
    veg_reflectivity_band_location1 = location1_veg_reflectivity_maps[band_index]

    plt.figure(figsize=(8, 6))
    plt.imshow(veg_reflectivity_band_location1, cmap='viridis')
    plt.colorbar(label='Reflectivity', shrink=0.2)
    plt.title(f'{band_name} Veg Reflectivity Map for Location 1')
    plt.axis('off')
    plt.show()


In [ ]:
#Create soil reflectivity map using the spatial and spectral features

import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio

# Directory containing TIFF images for all locations
image_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/clip locations stack/'

# Output directory for vegetation reflectivity maps
output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/soil reflectivity maps/'

# Create the output parent directory if it doesn't exist
os.makedirs(output_parent_directory, exist_ok=True)

# Initialize an empty list to store vegetation reflectivity maps for all locations
soil_reflectivity_maps = []

# Loop through all locations (assuming you have images from location1_stacked to location240_stacked)
for location_index in range(1, 241):
    location_name = f'location{location_index}'  # Location name based on the index
    
    # Specify the path to your cropped image for the current location
    location_stacked_path = os.path.join(image_directory, f'{location_name}_stacked.tif')

    # Open the cropped image using rasterio
    with rasterio.open(location_stacked_path) as src:
        cropped_image = src.read()
        crs = src.crs  # Get the CRS information from the metadata

    # Load the spectral reflectance signature of pure soil
    soil_endmember = all_average_soil_pixel_values[location_index - 1][1:]

    # Access the veg abundance map from the fractional abundance map
    soil_abundance_map = soil_abundance_maps[location_index - 1]  
    # Create a directory for vegetation reflectivity maps for the current location
    output_directory = os.path.join(output_parent_directory, location_name)

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    
    # Initialize an empty list to store reflectivity maps for the current location
    location_soil_reflectivity_maps = []

    # Loop through each band in your stacked image
    band_names = ['Blue', 'Green', 'Red', 'Red Edge', 'NIR']  # Replace with actual band names

    for band_index, band_name in enumerate(band_names):
        # Calculate the reflectivity for the current band using abundance and endmember information
        reflectivity_band_soil = soil_endmember[band_index] * soil_abundance_map
        
        # Append the reflectivity map to the list
        location_soil_reflectivity_maps.append(reflectivity_band_soil)

        # Plot and save the reflectivity map for the current band
        plt.figure(figsize=(8, 6))
        plt.imshow(reflectivity_band_soil, cmap='viridis')
        plt.colorbar(label='Reflectivity', shrink=0.2)
        plt.title(f'{band_name} Reflectivity Map for {location_name}')
        plt.axis('off')

        # Save the reflectivity map as an image
        output_filename = f'{location_name}_{band_name}_reflectivity_map.png'
        output_path = os.path.join(output_directory, output_filename)
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()
 
    # Append the reflectivity maps for the current location to the list of all maps
    soil_reflectivity_maps.append(location_soil_reflectivity_maps)


print("All soil reflectivity maps saved.")

In [ ]:
#sanity check for soil reflectivity map

# Access the reflectivity maps for location 1 from the list of all maps
location1_soil_reflectivity_maps = soil_reflectivity_maps[0]

# Check the shape of each reflectivity map for location 1
for band_index, reflectivity_map in enumerate(location1_soil_reflectivity_maps):
    print(f"Shape of soil Reflectivity Map for Location 1 - Band {band_index + 1}: {reflectivity_map.shape}")

# Plot the reflectivity maps for each band
band_names = ['Blue', 'Green', 'Red', 'Red Edge', 'NIR']  # Replace with actual band names

for band_index, band_name in enumerate(band_names):
    soil_reflectivity_band_location1 = location1_soil_reflectivity_maps[band_index]

    plt.figure(figsize=(8, 6))
    plt.imshow(soil_reflectivity_band_location1, cmap='viridis')
    plt.colorbar(label='Reflectivity', shrink=0.2)
    plt.title(f'{band_name} soil Reflectivity Map for Location 1')
    plt.axis('off')
    plt.show()


In [ ]:
#create an LUM reflectivity map 

import os
import numpy as np
import rasterio

# Output parent directory for stacked images
output_parent_directory = 'C:/Users/say70/Desktop/NX/2020/6_15/Processing pipeline/LUM stacked map/'

# Create the output parent directory if it doesn't exist
os.makedirs(output_parent_directory, exist_ok=True)

# Initialize an empty list to store stacked images
LUM_maps = []

# Loop through all locations (assuming you have 240 locations)
for location_index in range(1, 241):
    location_name = f'location{location_index}'  # Location name based on the index
    
    # Vegetation and soil reflectivity maps for the current location
    vegetation_maps = vegetation_reflectivity_maps[location_index - 1]
    soil_maps = soil_reflectivity_maps[location_index - 1]

    # Calculate the sum of vegetation and soil reflectivity bands
    stacked_image = np.stack([vegetation_maps[0] + soil_maps[0],
                              vegetation_maps[1] + soil_maps[1],
                              vegetation_maps[2] + soil_maps[2],
                              vegetation_maps[3] + soil_maps[3],
                              vegetation_maps[4] + soil_maps[4]], axis=0)
    
    # Print the shape of the stacked image
    #print(f"Shape of {location_name} stacked image:", stacked_image.shape)

    # Create a directory for the stacked image of the current location
    output_directory = os.path.join(output_parent_directory)
    os.makedirs(output_directory, exist_ok=True)

    # Save the stacked image as a TIFF file
    output_image_path = os.path.join(output_directory, f'{location_name}_stacked.tif')
    with rasterio.open(output_image_path, 'w', driver='GTiff', width=stacked_image.shape[2], height=stacked_image.shape[1], count=stacked_image.shape[0], dtype=stacked_image.dtype, crs=clipped_meta['crs'], transform=clipped_transform) as dst:
        dst.write(stacked_image)

    #print(f"Location {location_name} stacked image saved.")
     
    # Append the stacked image to the list
    LUM_maps.append(stacked_image)
    
print(f"LUM stacked maps image saved.")

In [ ]:
#Sanity check for the LUM maps

import matplotlib.pyplot as plt

# Location index for Location 1 (assuming it's the first location)
location_index = 1

# Select the stacked image for Location 1
location1_stacked_image = LUM_maps[location_index - 1]

# Get the band names (you may need to replace these with your actual band names)
band_names = ['Blue', 'Green', 'Red', 'Red Edge', 'NIR']

# Loop through each band and plot it
for band_index, band_name in enumerate(band_names):
    plt.figure(figsize=(8, 6))
    plt.imshow(location1_stacked_image[band_index], cmap='viridis')
    plt.colorbar(label='Reflectivity', shrink=0.2)
    plt.title(f'{band_name} Reflectivity Map for Location 1')
    plt.axis('off')
    plt.show()
